In [1]:
import json, requests
import pandas as pd
from tqdm import tqdm

In [2]:
# Load the SQuAD 2.0 JSON file
# URL of the SQuAD 2.0 dataset
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json'

# Fetch the content from the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load the JSON content into a dictionary
    squad_data = response.json()

In [3]:
# Initialize lists to hold flattened data
data_list = []

# Use tqdm to add a progress bar to the outer loop
for article in tqdm(squad_data['data'], desc='Processing Articles'):
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            qid = qa['id']
            is_impossible = qa.get('is_impossible', False)
            answers = qa.get('answers', [])
            if not is_impossible:
                for answer in answers:
                    answer_text = answer['text']
                    answer_start = answer['answer_start']
                    data_list.append({
                        'title': title,
                        'context': context,
                        'question': question,
                        'id': qid,
                        'is_impossible': is_impossible,
                        'answer_text': answer_text,
                        'answer_start': answer_start
                    })
            else:
                # For unanswerable questions, append with empty answer fields
                data_list.append({
                    'title': title,
                    'context': context,
                    'question': question,
                    'id': qid,
                    'is_impossible': is_impossible,
                    'answer_text': None,
                    'answer_start': None
                })

# Convert list of records to a DataFrame
df = pd.DataFrame(data_list)
df.set_index("id", inplace=True)

# Display the DataFrame
print(df.head())

Processing Articles: 100%|██████████| 442/442 [00:00<00:00, 4517.60it/s]

                            title  \
id                                  
56be85543aeaaa14008c9063  Beyoncé   
56be85543aeaaa14008c9065  Beyoncé   
56be85543aeaaa14008c9066  Beyoncé   
56bf6b0f3aeaaa14008c9601  Beyoncé   
56bf6b0f3aeaaa14008c9602  Beyoncé   

                                                                    context  \
id                                                                            
56be85543aeaaa14008c9063  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
56be85543aeaaa14008c9065  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
56be85543aeaaa14008c9066  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
56bf6b0f3aeaaa14008c9601  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
56bf6b0f3aeaaa14008c9602  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   

                                                                   question  \
id                                                                            
56be85543aeaaa14008c9063    

In [4]:
df.sample(5)

,title,context,question,is_impossible,answer_text,answer_start
id,,,,,,
5ad11b91645df0001a2d0dac,Uranium,Uranium is a naturally occurring element that ...,Where does uranium rank among elements in term...,True,None,NaN
57342c97d058e614000b6a9c,Portugal,"The 20th century saw the arrival of Modernism,...",In which century did Portugal see the arrival ...,False,20th century,4.0
56d20469e7d4791d009025fd,To_Kill_a_Mockingbird,Just as Lee explores Jem's development in comi...,Who chastis Scout for the way she dresse and a...,False,Mrs. Dubose,817.0
57290ee56aef051400154a07,States_of_Germany,"In 1952, following a referendum, Baden, Württe...",When did the German reunification take place?,False,1990,221.0
57324a59e17f3d14004227e4,Dwight_D._Eisenhower,French cooperation was deemed necessary to the...,Who served as High Commissioner of North Afric...,False,Henri Giraud,1072.0


Evaluation on LLM-RAG:

1. Retrieval - RAG retrieved the needed context? <br>
2. Relevance - LLM deciphered the relevant context? <br>
3. Accuracy - LLM answer contains the answer <br>
4. Precision - LLM answer does not contain irrelevant answer <br>







In [5]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

import deh.settings as settings
import deh.guardrail as guardrail
from deh.utils import format_context_documents as format_docs
from deh.utils import retriever_with_scores, dedupulicate_contexts
from deh.prompts import (
    qa_eval_prompt_with_context_text,
    LLMEvalResult,
    rag_prompt_llama_text,
    hyde_prompts,
)
import deh

In [6]:
LLM = Ollama(
            base_url=settings.OLLAMA_HOST, model=settings.LLM_MODEL, verbose=True
        )
print(LLM)

Ollama
Params: {'model': 'llama3.1:8b-instruct-q3_K_L', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}


/tmp/ipykernel_29/2248878434.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  LLM = Ollama(


In [7]:
row = 0
q = df.iloc[row]["question"]
c = df.iloc[row]["context"]
gt = df.iloc[row]["answer_text"]

# Initial LLM generation prompt:
qa_prompt = PromptTemplate(
    template=rag_prompt_llama_text, input_variables=["question", "context"]
)

chain = qa_prompt | LLM
llmans = chain.invoke({"question": q, "context": c})

print(llmans)

Beyoncé started becoming popular in the late 1990s as lead singer of R&B girl-group Destiny's Child. The group gained widespread recognition during this time, becoming one of the world's best-selling girl groups of all time. Her solo debut album, Dangerously in Love, was released in 2003 and further established her as a solo artist worldwide.


In [8]:
print(q)
print(c)
print(gt)
print(chain)

When did Beyonce start becoming popular?
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
in the late 1990s
first=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don'

In [9]:
import asyncio
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio

nest_asyncio.apply()

# Assuming chain.invoke is an asynchronous function
async def invoke_chain(chain, q, c, semaphore, pbar):
    async with semaphore:
        pbar.update(1)
        return await chain.ainvoke({"question": q, "context": c})

async def main(df = None, LLM = None, n=50, clim=10):
    # input validation 
    if df.empty:
        raise ValueError("The DataFrame is empty")
    assert (LLM != None)
    
    # Initial LLM generation prompt:
    qa_prompt = PromptTemplate(
        template=rag_prompt_llama_text, input_variables=["question", "context"]
    )
    chain = qa_prompt | LLM

    # Create a semaphore with a limit on concurrent tasks
    semaphore = asyncio.Semaphore(clim)

    # Create a tqdm progress bar
    samdf = df.sample(n)
    with tqdm(total=n, desc="Processing") as pbar:
        tasks = [invoke_chain(chain, row['question'], row['context'], semaphore, pbar) for _, row in samdf.iterrows()]
        results = await asyncio.gather(*tasks)

    return results

In [12]:
# Run the main function
results = await main(df, LLM, 1000, 32)

Processing: 100%|██████████| 1000/1000 [21:56<00:00,  1.32s/it]


In [13]:
# Print the results
for i in range(10): 
    print(str(i) + " : " + results[i])

0 : Afro Samurai initially aired on Spike TV channel in 2007. It was later adapted into a full feature animated film and became available on gaming consoles such as the PlayStation 3 and Xbox. The series was produced for American viewers by Studio Gonzo.
1 : An early form of semi-cursive script appeared at the end of the late Eastern Han period. It developed from a cursively written form of neo-clerical script. This new script is attributed to Liu Desheng, although it likely evolved over time through various masters.
2 : Pyometra affects unspayed female dogs of all types and ages. This condition can occur in any age group, not just older females. It is a serious medical issue that requires immediate attention.
3 : Two of the 50 largest cities in the United States are located in Oklahoma, which are Oklahoma City and Tulsa.
4 : Baruch College is the institution that described New York as the cultural capital of the world. This description was made in conjunction with diplomatic consulate